<a href="https://colab.research.google.com/github/ABEERAKHAN-01/Digit-Drawing-Recognition-Web-App/blob/main/Digit_Drawing_Recognition_Web_App_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***INTSALLING NECESSARY DEPENDENCIES***

In [1]:
# Install dependencies
!pip install -q streamlit-drawable-canvas
!pip install -q streamlit pyngrok tensorflow numpy pillow

import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow import keras

from pyngrok import ngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.7 MB/s eta 0:00:00


***USING NGROK FOR STREAMLIT CANVAS***

In [2]:
# Setup Ngrok


NGROK_AUTH_TOKEN = "31HcCiDVNBWzjR8AGpHLs38w76W_28jKUnWAvhD58dSevEyY2"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)


***LOADING SAVED MODEL***

In [3]:
# Load your uploaded model
MODEL_PATH = "mnist_dense.keras"

if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"{MODEL_PATH} not found! Please upload it to Colab /content/.")

# Load without optimizer/compile state
model = keras.models.load_model(MODEL_PATH, compile=False)
print("✅ Model loaded successfully:", MODEL_PATH)


✅ Model loaded successfully: mnist_dense.keras


***CREATING STREAMLIT APP***

In [4]:
%%writefile app.py
import os
import cv2
import numpy as np
import streamlit as st
from streamlit_drawable_canvas import st_canvas
import tensorflow as tf

st.set_page_config(page_title="Digit Recognition", page_icon="🖌️", layout="centered")

@st.cache_resource
def load_model():
    return tf.keras.models.load_model("mnist_dense.keras", compile=False)

model = load_model()

def preprocess(img_rgba):
    # RGBA → Gray
    gray = cv2.cvtColor(img_rgba.astype(np.uint8), cv2.COLOR_RGBA2GRAY)
    inv = 255 - gray  # invert → white digit on black background

    # Crop bounding box
    ys, xs = np.where(inv > 0)
    if len(xs) == 0 or len(ys) == 0:
        return np.zeros((28,28), np.float32)
    x1, x2, y1, y2 = xs.min(), xs.max(), ys.min(), ys.max()
    crop = inv[y1:y2+1, x1:x2+1]

    # Resize to 20x20, then pad to 28x28
    h, w = crop.shape
    if h > w:
        new_h, new_w = 20, int(w*(20/h))
    else:
        new_w, new_h = 20, int(h*(20/w))
    resized = cv2.resize(crop, (new_w, new_h))
    top, bottom = (28-new_h)//2, 28-new_h-(28-new_h)//2
    left, right = (28-new_w)//2, 28-new_w-(28-new_w)//2
    canvas = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0)
    return canvas.astype(np.float32)/255.0

def prepare_input(img28):
    shape = model.input_shape
    if len(shape)==2: # Dense (784,)
        return img28.reshape(1,784)
    else: # CNN (28,28,1)
        return img28.reshape(1,28,28,1)

st.title("🖌️ Digit Drawing Recognition")
st.write("Draw a digit below and get prediction:")

if "canvas_key" not in st.session_state:
    st.session_state.canvas_key = 0

canvas_result = st_canvas(
    stroke_color="#000000",
    stroke_width=15,
    background_color="#FFFFFF",
    height=280,
    width=280,
    drawing_mode="freedraw",
    key=f"canvas{st.session_state.canvas_key}"
)

if canvas_result.image_data is not None:
    img28 = preprocess(canvas_result.image_data)
    st.image((img28*255).astype(np.uint8), width=150, caption="28x28 Input")

    if st.button("Predict"):
        x = prepare_input(img28)
        preds = model.predict(x, verbose=0)[0]
        st.success(f"Prediction: {np.argmax(preds)} (Confidence {np.max(preds):.2%})")
        st.bar_chart({str(i): float(preds[i]) for i in range(10)})

if st.button("Clear / Try Again"):
    st.session_state.canvas_key += 1
    st.rerun()


Writing app.py


***GENERATING LINK FOR PREDICTION***

In [5]:
from pyngrok import ngrok
import subprocess, time, os, signal

# Kill previous if any
!pkill streamlit || echo "no streamlit running"

# Start streamlit
proc = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501", "--server.address", "0.0.0.0"])

# Wait and start tunnel
time.sleep(3)
public_url = ngrok.connect(8501).public_url
print("🌍 Your app is live at:", public_url)


no streamlit running
🌍 Your app is live at: https://47df1b0e1da9.ngrok-free.app
